In [2]:
import streamlit as st
import gspread as gs 

gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss_portfolio = gc.open_by_key(st.secrets['portfolio'].spreadsheet_key).worksheet(st.secrets['portfolio'].worksheet_name)

In [46]:
yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")['Close']

[*********************100%***********************]  2 of 2 completed


,AAPL,SPY
Date,,
2017-01-03,29.037500,225.240005
2017-01-04,29.004999,226.580002
2017-01-05,29.152500,226.399994
2017-01-06,29.477501,227.210007
2017-01-09,29.747499,226.460007
...,...,...
2017-04-24,35.910000,237.169998
2017-04-25,36.132500,238.550003
2017-04-26,35.919998,238.399994


In [17]:
import yfinance as yf
import pandas as pd

df_portfolio = pd.DataFrame(ss_portfolio.get_all_records())
financial = {
    'Asset': 

}
eurusd: pd.DataFrame = yf.download('EURUSD=X', start=min(df_portfolio['Date']))[['Close']]

df_portfolio['EURUSD'] = [eurusd[eurusd.index == df_portfolio.loc[idx]['Date']]['Close'].values[0] for idx in df_portfolio.index]
df_portfolio['Total€'] = [df_portfolio.loc[idx]['Total'] / (1 if df_portfolio.loc[idx]['Currency'] == "€" else df_portfolio.loc[idx]['EURUSD']) for idx in df_portfolio.index]
df_portfolio

[*********************100%***********************]  1 of 1 completed


,Date,Portfolio,Account,Asset,Price,Amount,Fees,Total,Total€,Currency,EURUSD,Out,Profit
0,2021-04-30,ZEN,eToro,Deposit$,0.8361,598.00000,,500.00,500.000000,€,1.212709,,
1,2021-04-30,ZEN,eToro,GREENBULL,1.0000,598.00000,,598.00,493.110802,$,1.212709,,
2,2021-07-30,ZEN,eToro,Deposit$,0.8361,598.00000,,500.00,500.000000,€,1.189300,,
3,2021-07-30,ZEN,eToro,GREENBULL,1.0000,585.00000,,585.00,491.886106,$,1.189300,,
4,2021-08-10,DMA,B.Direct PEA,Deposit€,1.0000,100.00000,,100.00,100.000000,€,1.173847,,
5,2021-08-10,ZEN,B.Direct CTO,Deposit€,1.0000,300.00000,,300.00,300.000000,€,1.173847,,
6,2021-10-01,ZEN,eToro,Deposit$,0.8696,575.00000,,500.00,500.000000,€,1.157448,,
7,2021-10-01,ZEN,eToro,GREENBULL,1.0000,575.00000,,575.00,496.782759,$,1.157448,,
8,2021-11-01,ZEN,eToro,Deposit$,0.8696,575.00000,,500.00,500.000000,€,1.155668,,
9,2021-11-01,ZEN,eToro,GREENBULL,1.0000,575.00000,,575.00,497.547525,$,1.155668,,


In [77]:
import numpy as np

df_deposit = df_portfolio[(df_portfolio['Portfolio'] == "ZEN REMIX") & (df_portfolio['Asset'].str.contains("Deposit"))]
df_deposit = df_deposit.groupby('Date').agg({'Total€': "sum"})
df_deposit['Deposit€'] = [np.sum(df_deposit.loc[:idx]['Total€']) for idx in df_deposit.index]
df_deposit = df_deposit.reset_index()[['Date', 'Deposit€']].astype({'Date': 'datetime64[ns]', 'Deposit€': 'int64'})
df_deposit

,Date,Deposit€
0,2021-04-30,493
1,2021-07-30,984
2,2021-08-10,1284
3,2021-10-01,1781
4,2021-11-01,2279
5,2021-12-01,3272
6,2022-02-01,3770
7,2022-03-24,4970
8,2022-04-01,7452
9,2022-04-19,9452


In [90]:

df_invested = df_portfolio[(df_portfolio['Portfolio'] == "ZEN REMIX") & (~df_portfolio['Asset'].str.contains("Deposit"))]
df_invested = df_invested.groupby('Date').agg({'Total€': "sum"})
df_invested['Invested€'] = [np.sum(df_invested.loc[:idx]['Total€']) for idx in df_invested.index]
df_invested = df_invested.reset_index()[['Date', 'Invested€']].astype({'Date': 'datetime64[ns]', 'Invested€': 'int64'})
df_invested

,Date,Invested€
0,2021-04-30,493
1,2021-07-30,984
2,2021-10-01,1481
3,2021-11-01,1979
4,2021-12-01,2517
5,2021-12-31,2972
6,2022-02-01,3471
7,2022-04-01,3966
8,2022-04-05,4447
9,2022-04-13,4869


In [38]:
import numpy as np
import datetime as dt

# Dataframe result
df = pd.DataFrame({'Date': pd.date_range(start=min(df_portfolio['Date']), end=dt.datetime.now())})

# Dataframe Initial
dfi: pd.DataFrame = df_portfolio.astype({'Date': 'datetime64[ns]'})

for portfolio in np.unique(dfi['Portfolio']):
    print(portfolio)
    # Dataframe of Portfolio
    dfp: pd.DataFrame = dfi[dfi['Portfolio'] == portfolio]

    for asset in np.unique(dfp['Asset']):
        print('  ', asset)
        # Dataframe of Asset
        dfa: pd.DataFrame = dfp[dfp['Asset'] == asset]
        dfa = dfa.groupby('Date').agg({'Total€': "sum", 'Amount': "sum"})

        amt, tot, pru = pd.Series(dtype=float), pd.Series(dtype=float), pd.Series(dtype=float)
        for idx in dfa.index:
            amt[idx] = np.sum(dfa.loc[:idx]['Amount'])
            tot[idx] = np.sum(dfa.loc[:idx]['Total€'])
            pru[idx] = tot[idx] / amt[idx]

        col = f'{portfolio}_{asset}'
        dfp = dfp.merge(pd.DataFrame({
            'Date': pd.to_datetime(dfa.index),
            f'{col}_Amount': amt,
            f'{col}_Total': tot,
            f'{col}_PRU': pru,
        }), on='Date', how='left').ffill()  

    cols_inv, cols_dep = [], []
    for col in dfp.columns:
        if portfolio in col and 'Total' in col:
            if 'Deposit' in col:
                cols_dep.append(col)
            else:
                cols_inv.append(col)

    dfp[f'{portfolio}_Invested'] = dfp[cols_inv].sum(axis=1)
    dfp[f'{portfolio}_Deposit'] = dfp[cols_dep].sum(axis=1)

    df = df.merge(dfp[
        ['Date'] + [col for col in dfp.columns if portfolio in col]
    ], on='Date', how='left').ffill() 


print('\n', df.shape, df.columns.to_list())
df.fillna(0)


DMA
   Deposit€
ZEN
   BTC
   Deposit$
   DepositUSDT
   Deposit€
   ESE
   GOLD
   GREENBULL

 (592, 29) ['Date', 'DMA_Deposit€_Amount', 'DMA_Deposit€_Total', 'DMA_Deposit€_PRU', 'DMA_Invested', 'DMA_Deposit', 'ZEN_BTC_Amount', 'ZEN_BTC_Total', 'ZEN_BTC_PRU', 'ZEN_Deposit$_Amount', 'ZEN_Deposit$_Total', 'ZEN_Deposit$_PRU', 'ZEN_DepositUSDT_Amount', 'ZEN_DepositUSDT_Total', 'ZEN_DepositUSDT_PRU', 'ZEN_Deposit€_Amount', 'ZEN_Deposit€_Total', 'ZEN_Deposit€_PRU', 'ZEN_ESE_Amount', 'ZEN_ESE_Total', 'ZEN_ESE_PRU', 'ZEN_GOLD_Amount', 'ZEN_GOLD_Total', 'ZEN_GOLD_PRU', 'ZEN_GREENBULL_Amount', 'ZEN_GREENBULL_Total', 'ZEN_GREENBULL_PRU', 'ZEN_Invested', 'ZEN_Deposit']


,Date,DMA_Deposit€_Amount,DMA_Deposit€_Total,DMA_Deposit€_PRU,DMA_Invested,DMA_Deposit,ZEN_BTC_Amount,ZEN_BTC_Total,ZEN_BTC_PRU,ZEN_Deposit$_Amount,...,ZEN_ESE_Total,ZEN_ESE_PRU,ZEN_GOLD_Amount,ZEN_GOLD_Total,ZEN_GOLD_PRU,ZEN_GREENBULL_Amount,ZEN_GREENBULL_Total,ZEN_GREENBULL_PRU,ZEN_Invested,ZEN_Deposit
0,2021-04-30,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,598.0,...,0.0,0.000000,0.0,0.00,0.000,598.0,493.110802,0.824600,493.110802,500.000000
1,2021-04-30,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,598.0,...,0.0,0.000000,0.0,0.00,0.000,598.0,493.110802,0.824600,493.110802,500.000000
2,2021-05-01,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,598.0,...,0.0,0.000000,0.0,0.00,0.000,598.0,493.110802,0.824600,493.110802,500.000000
3,2021-05-02,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,598.0,...,0.0,0.000000,0.0,0.00,0.000,598.0,493.110802,0.824600,493.110802,500.000000
4,2021-05-03,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,598.0,...,0.0,0.000000,0.0,0.00,0.000,598.0,493.110802,0.824600,493.110802,500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,2022-11-19,8500.0,8500.0,1.0,0.0,8500.0,0.05524,1468.700241,26587.622028,5109.0,...,986.4,17.934545,26.0,1815.19,69.815,5028.0,4459.811596,0.886995,8730.101837,11965.805016
588,2022-11-20,8500.0,8500.0,1.0,0.0,8500.0,0.05524,1468.700241,26587.622028,5109.0,...,986.4,17.934545,26.0,1815.19,69.815,5028.0,4459.811596,0.886995,8730.101837,11965.805016
589,2022-11-21,8500.0,8500.0,1.0,0.0,8500.0,0.05524,1468.700241,26587.622028,5109.0,...,986.4,17.934545,26.0,1815.19,69.815,5028.0,4459.811596,0.886995,8730.101837,11965.805016
590,2022-11-22,8500.0,8500.0,1.0,0.0,8500.0,0.05524,1468.700241,26587.622028,5109.0,...,986.4,17.934545,26.0,1815.19,69.815,5028.0,4459.811596,0.886995,8730.101837,11965.805016


In [36]:
a = b = []
print('a', a)
print('b', b)
a.append(10)
print('a', a)
print('b', b)

a []
b []
a [10]
b [10]


In [79]:
import datetime as dt
df = pd.DataFrame({'Date': pd.date_range(start=min(df_portfolio['Date']), end=dt.datetime.now())})

df = df.merge(df_invested.reset_index()[['Date', 'Invested€']].astype({'Date': 'datetime64[ns]', 'Invested€': 'int64'}), on='Date', how='left').ffill()
df = df.merge(df_deposit.reset_index()[['Date', 'Deposit€']].astype({'Date': 'datetime64[ns]', 'Deposit€': 'int64'}), on='Date', how='left').ffill()
# df = df.merge(df_invested.reset_index().astype({'Date': 'datetime64[ns]'})['Invested€'], how='left', on='Date').ffill()
# df = df.merge(df_deposit['Deposit€'], how='left', left_index=True, right_index=True).ffill()
# df = df.merge(df_invested[['Date']], how='left', left_index=True, right_on='Date')
# df = pd.merge(pd.DataFrame(index=pd.date_range(start=min(df_portfolio['Date']), end=max(df_portfolio['Date']))), df_deposit['Date'].astype('datetime64[ns]'), left_index=True, right_on='Date')
df

,Date,Invested€,Deposit€
0,2021-04-30,493.0,493.0
1,2021-05-01,493.0,493.0
2,2021-05-02,493.0,493.0
3,2021-05-03,493.0,493.0
4,2021-05-04,493.0,493.0
...,...,...,...
567,2022-11-18,8730.0,11946.0
568,2022-11-19,8730.0,11946.0
569,2022-11-20,8730.0,11946.0
570,2022-11-21,8730.0,11946.0


In [81]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Deposit€']))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Invested€']))
fig.show()